In [1]:
#!/usr/bin/python3
"""
Logic grid puzzle: 'origin' in CPpy

Based on... to check originally, currently part of ZebraTutor
Probably part of Jens Claes' master thesis, from a 'Byron...' booklet
"""
import sys
sys.path.append('/home/crunchmonster/Documents/VUB/01_SharedProjects/01_cppy_src')
from cppy import *
import numpy
import pandas as pd

In [2]:
# Relation between 'rows' and 'cols', Boolean Variables in a pandas dataframe
class Relation(object):
    # rows, cols: list of names
    def __init__(self, rows, cols):
        rel = BoolVar((len(rows),len(cols)))
        self.df = pd.DataFrame(index=rows, columns=cols)
        for i,r in enumerate(rows):
            for j,c in enumerate(cols):
                self.df.loc[r,c] = rel[i,j]
    # use as: rel['a','b']
    def __getitem__(self, key):
        try:
            return self.df.loc[key]
        except KeyError:
            return False

In [3]:
person = ['Mattie', 'Ernesto', 'Roxanne', 'Zachary', 'John']
age = ['109', '110', '111', '112', '113']
city = ['Brussels', 'Tehama', 'Zearing', 'Plymouth', 'Shaver Lake']
birthplace = ['Mexico', 'Oregon', 'Kansas', 'Washington', 'Alaska']

types = [person, age, city, birthplace]
n = len(types)
m = len(types[0])
assert all(len(types[i]) == m for i in range(n)), "all types should have equal length"

is_old = Relation(person, age)
lives_in = Relation(person, city)
native = Relation(person, birthplace)
age_city = Relation(age, city)
age_birth = Relation(age, birthplace)
city_birth = Relation(city, birthplace)

In [4]:
# Bijectivity
bij = []
for rel in [is_old, lives_in, native, age_city, age_birth, city_birth]:
    # for each relation
    for col in rel.df:
        # one per column
        bij.append( sum(rel[:,col]) == 1 )
    for (_,row) in rel.df.iterrows():
        # one per row
        bij.append( sum(row) == 1 )


# Transitivity
trans = []
for p in person:
    for c in city:
        trans.append( [implies(is_old[p,a] & age_city[a,c],
                                  lives_in[p,c]) for a in age] )
    for b in birthplace:
        trans.append( [implies(is_old[p,a] & age_birth[a,b],
                                  native[p,b]) for a in age] )
        trans.append( [implies(lives_in[p,c] & city_birth[c,b],
                                  native[p,b]) for c in city] )
for a in age:
    for b in birthplace:
        trans.append( [implies(age_city[a,c] & city_birth[c,b],
                                  age_birth[a,b]) for c in city] )


# Clues
clues = []
# Mattie is 113 years old
clues.append( is_old['Mattie', '113'] )

# The person who lives in Tehama is a native of either Kansas or Oregon
clues.append( [implies(lives_in[p,'Tehama'],
                       native[p,'Kansas'] | native[p,'Oregon']) for p in person] )

# The Washington native is 1 year older than Ernesto
clues.append( [implies(age_birth[a,'Washington'],
                       is_old['Ernesto',str(int(a)-1)]) for a in age] )

# Roxanne is 2 years younger than the Kansas native
clues.append( [implies(is_old['Roxanne',a], 
                       age_birth[str(int(a)+2), 'Kansas']) for a in age] )

# The person who lives in Zearing isn't a native of Alaska
clues.append( [implies(lives_in[p,'Zearing'],
                       ~native[p,'Alaska']) for p in person] )

# The person who is 111 years old doesn't live in Plymouth
clues.append( [implies(is_old[p,'111'],
                       ~lives_in[p,'Plymouth']) for p in person] )

# The Oregon native is either Zachary or the person who lives in Tehama
clues.append( [implies(native[p,'Oregon'],
                       (p == 'Zachary') | lives_in[p,'Tehama']) for p in person] )

# The person who lives in Shaver Lake is 1 year younger than Roxanne
clues.append( [implies(age_city[a,'Shaver Lake'],
                       is_old['Roxanne',str(int(a)+1)]) for a in age] )

# The centenarian who lives in Plymouth isn't a native of Alaska
clues.append( [implies(lives_in[p,'Plymouth'],
                       ~native[p,'Alaska']) for p in person] )

# Of the person who lives in Tehama and Mattie, one is a native of Alaska and the other is from Kansas
clues.append( [implies(lives_in[p,'Tehama'],
                       (p != 'Mattie') &
                       ((native['Mattie','Alaska'] & native[p,'Kansas']) |
                        (native[p,'Alaska'] & native['Mattie','Kansas']))) for p in person] )

In [9]:
trans

[[((BV0) and (BV75)) -> (BV25),
  ((BV1) and (BV80)) -> (BV25),
  ((BV2) and (BV85)) -> (BV25),
  ((BV3) and (BV90)) -> (BV25),
  ((BV4) and (BV95)) -> (BV25)],
 [((BV0) and (BV76)) -> (BV26),
  ((BV1) and (BV81)) -> (BV26),
  ((BV2) and (BV86)) -> (BV26),
  ((BV3) and (BV91)) -> (BV26),
  ((BV4) and (BV96)) -> (BV26)],
 [((BV0) and (BV77)) -> (BV27),
  ((BV1) and (BV82)) -> (BV27),
  ((BV2) and (BV87)) -> (BV27),
  ((BV3) and (BV92)) -> (BV27),
  ((BV4) and (BV97)) -> (BV27)],
 [((BV0) and (BV78)) -> (BV28),
  ((BV1) and (BV83)) -> (BV28),
  ((BV2) and (BV88)) -> (BV28),
  ((BV3) and (BV93)) -> (BV28),
  ((BV4) and (BV98)) -> (BV28)],
 [((BV0) and (BV79)) -> (BV29),
  ((BV1) and (BV84)) -> (BV29),
  ((BV2) and (BV89)) -> (BV29),
  ((BV3) and (BV94)) -> (BV29),
  ((BV4) and (BV99)) -> (BV29)],
 [((BV0) and (BV100)) -> (BV50),
  ((BV1) and (BV105)) -> (BV50),
  ((BV2) and (BV110)) -> (BV50),
  ((BV3) and (BV115)) -> (BV50),
  ((BV4) and (BV120)) -> (BV50)],
 [((BV25) and (BV125)) -> (BV

In [ ]:
model = Model([bij, trans, clues])
stats = model.solve(solver = "ortools") #solver="minizinc_text")

for p in person:
    lst = [p]
    for rel,other in zip([is_old, lives_in, native], [age, city, birthplace]):
        for x in other:
            if rel[p,x].value():
                lst.append(x)
    print(",".join(lst))